In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift

In [ ]:
df = pd.read_csv('checkins.dat', sep='|')

#в исходных названиях столбцов лишние пробелы, удалим их
df.rename(columns=lambda x: x.strip(), inplace=True)

In [3]:
#заполним пустые значения (там whitespaces) на NaN
df.loc[:, ['latitude', 'longitude']] = df.loc[:, ['latitude', 'longitude']].apply(pd.to_numeric, errors='coerce')

#!!!можно сразу так data = pd.read_csv('checkins.dat', sep='|', skipinitialspace=True)
#но с именами колонок все равно придется поработать

#удалим строки, где в любом из указанных столбцов есть NaN
df = df.dropna(axis=0, subset=['latitude', 'longitude'], how='any')

df.reset_index(inplace=True,drop=True)

In [4]:
df.shape

(396634, 6)

In [5]:
#на всем датасете долго, на 100к строк сильно быстрее, так говорится в описании условия задачи
new_df = df.iloc[:10**5]
X = new_df.loc[:,['latitude', 'longitude']].values

In [6]:
clustering = MeanShift(bandwidth=0.1).fit(X)

In [7]:
len(np.unique(clustering.labels_))

3231

In [71]:
#отбросим кластеры, где меньше 16 точек
uniqelem_and_counts = np.unique(clustering.labels_, return_counts=True)
big_clusters = [x[0] for x in list(zip(uniqelem_and_counts[0],uniqelem_and_counts[1])) if x[1] > 15]

In [72]:
centers_labels = [x for x in list(zip(clustering.cluster_centers_, clustering.labels_)) if x[1] in big_clusters]

In [73]:
#координаты офисов из задания
coordinates_offices = {'LA':(33.751277, -118.188740), 'Miami':(25.867736, -80.324116),
                       'London':(51.503016, -0.075479), 'Amster':(52.378894, 4.885084),
                       'Beijing':(39.366487, 117.036146), 'Sydney':(-33.868457, 151.205134)}

In [74]:
#предсказание ближайшего кластера к каждому из центров
for k, v in coordinates_offices.items():
    v1, v2 = v
    print('Ближайший кластер(скопление туристов) - {1} к офису в {0} '.format(k,clustering.predict([[v1,v2]])[0]))

Ближайший кластер(скопление туристов) - 51 к офису в LA 
Ближайший кластер(скопление туристов) - 419 к офису в Miami 
Ближайший кластер(скопление туристов) - 58 к офису в London 
Ближайший кластер(скопление туристов) - 370 к офису в Amster 
Ближайший кластер(скопление туристов) - 1980 к офису в Beijing 
Ближайший кластер(скопление туристов) - 420 к офису в Sydney 


In [75]:
dist_to_offices = {'LA':[], 'Miami':[], 'London':[], 'Amster':[], 'Beijing':[], 'Sydney':[] }
for centr, lab in centers_labels:
    dist = []
    for key, xy_office in coordinates_offices.items():
        dist.append((key, np.linalg.norm(xy_office - centr)))
    near_office = min(dist, key=lambda t: t[1])
    dist_to_offices[near_office[0]].append((near_office[1], centr, lab))
for k, v in dist_to_offices.items():
    v.sort(key=lambda t: t[0])  

In [76]:
c = 20
nearest_offices = []
for k, v in dist_to_offices.items():
    for i, vv in enumerate(v):
        if i == 20:
            break
        nearest_offices.append((k,vv))
nearest_offices.sort(key=lambda t: t[1])

In [77]:
#расспечатаем ближайший офис к кластеру
print(nearest_offices[0])

('Sydney', (0.007834758163107856, array([-33.86063043, 151.20477593]), 31))


In [78]:
dist_to_offices[nearest_offices[0][0]][0]

(0.007834758163107856, array([-33.86063043, 151.20477593]), 31)